In [1]:
from transformers import BertConfig, BertTokenizerFast, BertForSequenceClassification
from transformers import AutoConfig, AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback

model_path = 'abhi1nandy2/Bible-roberta-base'

config = AutoConfig.from_pretrained(model_path, num_labels=1, dropout=0.3, attention_dropout=0.3)
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("abhi1nandy2/Bible-roberta-base")
model = AutoModelForSequenceClassification.from_pretrained("abhi1nandy2/Bible-roberta-base", config = config)

2024-06-07 18:14:31.456640: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-07 18:14:31.456731: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-07 18:14:31.567144: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/567 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at abhi1nandy2/Bible-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
model

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [3]:
!wget -O train.tsv "https://raw.githubusercontent.com/neilrs123/Lexical-Complexity-Prediction/master/Dataset/Sub-task%201/lcp_single_train.tsv"
!wget -O trial.tsv "https://raw.githubusercontent.com/neilrs123/Lexical-Complexity-Prediction/master/Dataset/Sub-task%201/lcp_single_trial.tsv"

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


--2024-06-07 18:14:54--  https://raw.githubusercontent.com/neilrs123/Lexical-Complexity-Prediction/master/Dataset/Sub-task%201/lcp_single_train.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1746979 (1.7M) [text/plain]
Saving to: 'train.tsv'

train.tsv           100%[===================>]   1.67M  --.-KB/s    in 0.02s   

2024-06-07 18:14:54 (72.6 MB/s) - 'train.tsv' saved [1746979/1746979]

--2024-06-07 18:14:57--  https://raw.githubusercontent.com/neilrs123/Lexical-Complexity-Prediction/master/Dataset/Sub-task%201/lcp_single_trial.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connec

In [4]:
import pandas as pd
train = pd.read_table("/kaggle/working/train.tsv")
# train = train[train["corpus"] == "bible"].drop(["corpus", "id"], axis = 1)

trial = pd.read_table("/kaggle/working/trial.tsv")
# trial = trial[trial["subcorpus"] == "bible"].drop(["subcorpus", "id"], axis = 1)

train_data = train[:-100]
trial = trial[:35].reset_index(drop = True)

In [5]:
from datasets import Dataset

train_data = Dataset.from_pandas(train_data)
# eval_data = Dataset.from_pandas(eval_data)
eval_data = Dataset.from_pandas(trial)

In [7]:
# Preprocess function: sentence + [SEP] + token
def preprocess_function(examples):
    sentences = [f"{s} [SEP] {t}" for s, t in zip(examples['sentence'], examples['token'])]
    tokenized_inputs = tokenizer(sentences, padding=True, truncation=True, max_length = 256)
    tokenized_inputs['labels'] = examples['complexity']
    return tokenized_inputs

# Map the preprocess function to datasets
train_dataset = train_data.map(preprocess_function, batched=True, batch_size=512)
eval_dataset = eval_data.map(preprocess_function, batched=True, batch_size=16)

Map:   0%|          | 0/7132 [00:00<?, ? examples/s]

Map:   0%|          | 0/35 [00:00<?, ? examples/s]

In [8]:
from torch import cuda
import torch
device = 'cuda' if cuda.is_available() else 'cpu'

In [9]:
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(
    output_dir='/kaggle/working/model',
    report_to="none",
    do_train=True,
    do_eval=True,
    num_train_epochs=10,
    warmup_steps=100,
    weight_decay=0.01,
    logging_strategy='steps',
    logging_dir='./logs',
    logging_steps=50,
    evaluation_strategy="steps",
    save_strategy="steps",
    fp16=torch.cuda.is_available(),
    load_best_model_at_end=True,
    lr_scheduler_type='linear',  # Adjust learning rate scheduling
    learning_rate=2e-5,
)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [10]:
import numpy as np
from scipy.stats import pearsonr
def compute_metrics(pred):
    preds = np.squeeze(pred.predictions)
    return {
            "Pearson" : pearsonr(preds,pred.label_ids)[0],
           }

In [11]:
trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        compute_metrics=compute_metrics,
        tokenizer=tokenizer
)

In [12]:
import warnings
warnings.filterwarnings("ignore")

In [13]:
train_result = trainer.train()

Step,Training Loss,Validation Loss,Pearson
50,0.063900,0.022670,0.429875
100,0.024400,0.029517,0.292197
150,0.022900,0.023353,0.176161
200,0.017600,0.024343,0.266266
250,0.016500,0.010971,0.257270
300,0.014800,0.011285,0.329752
350,0.014100,0.009582,0.185602
400,0.014800,0.010853,0.234796
450,0.012300,0.016503,0.142127
500,0.014700,0.014364,0.111849


KeyboardInterrupt: 